In [1]:
import pandas as pd
import os
import re
import romkan
import unicodedata

In [2]:
# get irregular words from ../data/irregular/irregular.csv and create a dict to avoid those words

In [3]:
vocab = pd.read_csv('../data/created/JMdict_e.csv', sep='\t')
kanji = pd.read_csv('../data/created/kanjidic2.csv')

In [4]:
# Run once
del vocab['pos']
del vocab['meaning']
del kanji['Unnamed: 0']
vocab.pronunciation = vocab.pronunciation.map(lambda x: romkan.to_hiragana(romkan.to_roma(x)))
kanji.on = kanji.on.map(lambda x: re.findall("""[\u3040-\u309f]+""", romkan.to_hiragana(romkan.to_roma(x))))
##############################
#kanji.kun = kanji.kun.map(lambda x: list(set(map(lambda y: y.split('.')[0],re.findall("""[\u3040-\u309f.]+""", x))))) # THIS THREW AWAY GOOD INFO
##############################
kanji['kun1'] = kanji.kun.map(lambda x: list(set(map(lambda y: y.split('.')[0],re.findall("""[\u3040-\u309f.]+""", x)))))
kanji['kun2'] = kanji.kun.map(lambda x: list(set(map(lambda y: ''.join(y.split('.')),re.findall("""[\u3040-\u309f.]+""", x)))))
# I should also include the kun without splitting somehow... aka shi.ku = shi and shiku
kanji.nanori = kanji.nanori.map(lambda x: re.findall("""[\u3040-\u309f]+""", x))
kanji.meaning = kanji.meaning.map(lambda x: re.findall("""[a-zA-Z-]+ *[a-zA-Z-]+""", x))
#char_ser = pd.Series(kanji.on.values,index=kanji.character).to_dict()
kanji['onkun'] = kanji.kun1 + kanji.kun2 + kanji.on
#kanji.onkun.map(lambda x: list(set(x))) ########## NOW MAKE SURE ALL ARE SET TO LISTS SO THERE ARE NO DUPLICATES
char_ser = pd.Series(kanji.onkun.values,index=kanji.character).to_dict()
kanji.head()

,character,freq,meaning,on,kun,nanori,kun1,kun2,onkun
0,亜,1509.0,"[Asia, rank next, come after, -ous]",[あ],['つ.ぐ'],"[や, つぎ, つぐ]",[つ],[つぐ],"[つ, つぐ, あ]"
1,唖,NaN,"[mute, dumb]","[あ, あく]",['おし'],[],[おし],[おし],"[おし, おし, あ, あく]"
2,娃,NaN,[beautiful],"[あ, あい, わ]",['うつく.しい'],[い],[うつく],[うつくしい],"[うつく, うつくしい, あ, あい, わ]"
3,阿,1126.0,"[Africa, flatter, fawn upon, corner, nook, rec...","[あ, お]","['おもね.る', 'くま']","[ほとり, あず, あわ, おか, きた, な]","[くま, おもね]","[おもねる, くま]","[くま, おもね, おもねる, くま, あ, お]"
4,哀,1715.0,"[pathetic, grief, sorrow, pathos, pity, sympat...",[あい],"['あわ.れ', 'あわ.れむ', 'かな.しい']",[],"[かな, あわ]","[あわれ, かなしい, あわれむ]","[かな, あわ, あわれ, かなしい, あわれむ, あい]"


In [5]:
#vocab.vocab = vocab.vocab.map(lambda x: romkan.to_hiragana(romkan.to_roma(x)))
#vocab.head()

#char_ser['亜']

#print(jlpt[jlpt.Kanji.str.contains('風')])
#print(char_ser['風'])

In [6]:
irregular = pd.read_csv('../data/irregular/irregular.csv')
jlpt = pd.read_csv('../data/vocab/jlptvocab.csv')
del irregular['URL']

In [7]:
irregular.head()

,Link
0,愛斯蘭
1,銅
2,欠伸
3,飛魚
4,麻布


In [8]:
jlpt = jlpt[jlpt.Kanji.isnull() == 0]
jlpt = jlpt[jlpt.Hiragana.isnull() == 0]
jlpt = jlpt[jlpt.English.isnull() == 0]
jlpt.tail()

,Kanji,Hiragana,English
8334,忘れ物,わすれもの,lost article
8335,笑う,わらう,"to laugh,to smile"
8336,割合,わりあい,"rate,ratio,percentage"
8337,割れる,われる,to break
8353,消しゴム,けしゴム,"eraser, rubber"


In [9]:
def unvoice(char):
    if not ('\u3041' <= char <= '\u3096') or len(char) > 1 or not unicodedata.decomposition(char):
        return char
    return unicodedata.normalize('NFD', char)[0]

In [10]:
### DONT RUN ###
#def check_word(word,cur,pron,pastprons,pastpron):
#    if cur > len(word):
#        return pron
#    try:
#        prons = set(char_ser[word[cur]])
#    except:
#        print(word, ' is not in the dictionary')
#    
#    past_prons.append('')
####

In [11]:
# if prons[-1] == ん: prons = [prons,prons[:-1]]
# if 
def recurs(kanji, word, pron_list = [], done = False):
    if len(pron_list) == len(kanji) or len(word) == 0: # check for 々 and repeat past syllable (unsounded)
        return pron_list, False
    cur = kanji[len(pron_list)]
    if re.match("""[\u3040-\u30ff]""", cur) != None: #should we romkan to hiragana?? ALSO HYPHEN
        if cur != word[0]:
            return pron_list, False
        prons = cur
    elif cur == '々':
        if unvoice(pron_list[-1]) == unvoice(word[:len(pron_list[-1])]): # or 
            pron_list.append(word[:len(pron_list[-1])])
            return pron_list, len(kanji) == len(pron_list)
        else:
            return None, False
    else:
        prons = [char_ser[cur]] # .sort(key=len, reverse=True)
    #prons = set(prons)
    for pron in prons:        # GET RID OF THIS?
        if pron[-1] == ん:
            prons.append[pron[:-1]]
    for pron in prons:
        wordmatch = word[:len(pron)]
        wordmatch = unvoice(wordmatch[0]) + wordmatch[1:]
        words = [wordmatch, word[:len(pron)]]
        if pron in words:
            pron_list.append(word[:len(pron)])
            word = word[len(pron):]
            if len(pron_list) == len(kanji) and len(word) == 0:
                return pron_list, True
            new_pron_list, done = recurs(kanji,word,pron_list,done)
            if done:
                return new_pron_list, done
            word = pron_list.pop() + word
    return None, False

# update with unvoice the first syllable of word


# PRONUNCIATION DOES NOT MATCH ON or KUN YOMIKATA... MOSTLY DROPPING LAST N??? -------------

In [12]:
#char_ser['車']
#'boy'[1:]
#recurs('相変わらず','あいかわらず',[])
char_ser['一']#.sort(key=len, reverse=True)    # EITHER INCLUDE NANORI OR FIX THE ABOVE

['ひと', 'ひとつ', 'ひと', 'いち', 'いつ']

In [13]:
true_cnt = 0
false_cnt = 0
for _,line in jlpt.iterrows():
    kanji = line.Kanji
    word = line.Hiragana
    print(kanji,word)
    a,b = recurs(kanji,word,[])
    print(a,b)
    if a == None:
        false_cnt += 1
    else:
        true_cnt += 1
print(true_cnt, false_cnt) #~8% are irregular...

嗚呼 ああ


NameError: name 'ん' is not defined

In [103]:
#jlpt[jlpt.Kanji.str.contains('亜')]
#jlpt[jlpt.Kanji.isnull()]
print(recurs('十一人','じゅういちにん',[]))
print(recurs('一人','ひとり',[]))
#print(jlpt)

(['じゅう', 'いち', 'にん'], True)
(['ひと', 'り'], True)


In [45]:
char_on = pd.Series(kanji.on.values,index=kanji.character).to_dict()
char_kun = pd.Series(kanji.kun.values,index=kanji.character).to_dict()

In [96]:
#jlpt[jlpt.Kanji.str.contains('亜')]
jlpt.head()

,Kanji,Hiragana,English
0,嗚呼,ああ,"Ah!, Oh!, Alas!"
1,相,あい,"together, mutually, fellow"
2,相変わらず,あいかわらず,"as ever, as usual, the same"
3,愛想,あいそ,"civility, courtesy, compliments, sociability, ..."
4,相対,あいたい,"confrontation, facing, between ourselves, no t..."


In [92]:
### THIS WAS A PREVIOUS TEST
word = '漢数字'
pron = 'かんすうじ'
for ji in word:
    pron_list = re.findall("""[\u3040-\u309f]+""", char_ser[ji])
    pron_list.sort(key=len, reverse=True)
    print(pron_list)
    for pr in pron_list:
        if pron[0:len(pr)] == pr: # re match beginning of string
            print(ji, pr)
            pron = pron[len(pr):]
    #re.match()
    
#next match up each kanji to on or kun for exploratory analysis

#Maybe if word is only kanji - first check on list, if word is mixture - first check kun?

# Also put in checks for hirgana matching ()

['かん']
漢 かん
['すう', 'さく', 'そく', 'しゅ', 'す']
数 すう
['じ']
字 じ
